# Clean Median House Price Data

In [1]:
# Import the required libraries
import requests
import json
import pandas as pd


In [2]:
#Import median house price by suburb data
#encodings = ['utf-8', 'utf-8-sig', 'latin-1', 'cp1252', 'utf-16']

# Specify the file path and sheet name
file_path = "Resources\Copy-of-Suburb_HouseV2022z.xlsx"
sheet_name = "Sheet1"

# Read the Excel file, specifying the sheet name and header row
house_prices_df = pd.read_excel(file_path, sheet_name, header=[1])

# Remove the first row of data
house_prices_df  = house_prices_df.drop(0)

house_prices_df= house_prices_df.dropna(subset=[2022])

new_column_names = {
    "Unnamed: 0": "HOUSE_LOCALITY",
    "change": "change 2021-22",
    "change.1":"change 2021-22"
    }
house_prices_df = house_prices_df.rename(columns=new_column_names)
house_prices_df

,HOUSE_LOCALITY,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,prelim 2023,change 2021-22,change 2021-22,Growth PA
3,ABBOTSFORD,714000,792500,862500,925000,1187500,1280000,1192500,1050000,1200000,1365000,1341500,1409000,-2,88,6.5
4,ABERFELDIE,852500,947500,1045000,1207500,1300000,1471000,1498500,1390000,1520000,1858000,1860000,NaN,0,118,8.1
5,AINTREE,-,-,-,600000,600000,571000,557500,575000,630000,719000,760000,710000,6,NaN,NaN
6,AIREYS INLET,634000,664000,625500,680000,715000,737500,869000,985000,1132500,1775000,1725000,1570000,-3,172,10.5
7,AIRPORT WEST,495000,532000,575000,635000,742000,845000,845000,795000,812500,965000,926500,795000,-4,87,6.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
824,YARRAM,215000,190000,202500,245000,230000,225000,218000,240000,260000,360000,462500,387500,28,115,8.0
825,YARRAVILLE,613000,696000,710000,810000,900500,1000000,1045000,971000,1033000,1195000,1205000,1070000,1,97,7.0
826,YARRAWONGA,300000,302500,297000,300500,320000,324500,345000,375000,456500,565000,665000,591000,18,122,8.3
827,YEA,255000,260000,255000,260000,305000,385000,390000,435500,422500,570000,617500,420000,8,142,9.2


In [3]:
# Check for NaN values in a column
has_nan = house_prices_df['HOUSE_LOCALITY'].isna().any()

# Display the result - False (no NaN values in column), True (at least one NaN value in column)
print(has_nan)

False


In [4]:
#Import mapping file - use to filter to sample postcodes for further analysis
# Specify the file path and sheet name
file_path = "Resources/Melbourne Postcodes.xlsx"
sheet_name = "Mapping"

# Read the specified sheet into a DataFrame
melb_postcodes_df = pd.read_excel(file_path, sheet_name=sheet_name)
melb_postcodes_df= melb_postcodes_df.dropna(subset=['ABS_SA2_KEY'])
melb_postcodes_df= melb_postcodes_df.dropna(subset=['HOUSE_LOCALITY'])
melb_postcodes_df= melb_postcodes_df.dropna(subset=['SCHOOL_POST_CODE'])

# Display the DataFrame
melb_postcodes_df

,MUNICIPALITY,CITY_SHIRE,SUBURB_GROUP,RURAL_TOWNSHIP,POST_CODE,ABS_SA2_KEY,HOUSE_LOCALITY,SCHOOL_POST_CODE,SUBURB_NAME,COMMENTS,SUBURB_POSTCODE_COMMENTS
0,Inner City municipalities and their suburbs,City of Melbourne,Inner,NaN,3053,206041117: Carlton,CARLTON,3053.0,Carlton,NaN,Carlton 3053
2,Inner City municipalities and their suburbs,City of Yarra,Inner,NaN,3054,206071140: Carlton North - Princes Hill,CARLTON NORTH,3054.0,Carlton North,Shared with City of Yarra,Carlton North 3054 (Shared with City of Yarra)
6,Northern municipalities and their suburbs,City of Moonee Valley,Mid,NaN,3031,206031115: Flemington,FLEMINGTON,3031.0,Flemington,Shared with City of Moonee Valley,Flemington 3031 (Shared with City of Moonee Va...
7,Northern municipalities and their suburbs,City of Moonee Valley,Mid,NaN,3031,206031115: Flemington,KENSINGTON,3031.0,Kensington,NaN,Kensington 3031
10,Inner City municipalities and their suburbs,City of Melbourne,Inner,NaN,3051,206041506: North Melbourne,NORTH MELBOURNE,3051.0,North Melbourne,Shared with City of Moonee Valley,North Melbourne 3051 (Shared with City of Moon...
...,...,...,...,...,...,...,...,...,...,...,...
997,Western municipalities and their suburbs,City of Wyndham,Outer,NaN,3030,213011570: Derrimut,WERRIBEE,3030.0,Werribee,NaN,Werribee 3030
1000,Western municipalities and their suburbs,City of Wyndham,Outer,NaN,3030,213011570: Derrimut,WERRIBEE SOUTH,3030.0,Werribee South,NaN,Werribee South 3030
1003,Western municipalities and their suburbs,City of Wyndham,Outer,NaN,3024,213051579: Manor Lakes - Quandong,WYNDHAM VALE,3024.0,Wyndham Vale,NaN,Wyndham Vale 3024
1005,Western municipalities and their suburbs,City of Melton,Outer,Rural localities,3338,213041571: Brookfield,EYNESBURY,3338.0,Eynesbury,Shared with the Shire of Melton,Eynesbury 3338 (Shared with the Shire of Melton)


In [5]:
# Concatenate SUBURB_NAME values for each POST_CODE
postcode_suburbs_df = melb_postcodes_df.groupby('POST_CODE')['SUBURB_NAME'].agg(lambda x: ', '.join(x)).reset_index()

# Display the resulting DataFrame
postcode_suburbs_df

,POST_CODE,SUBURB_NAME
0,3003,West Melbourne
1,3011,"Footscray, Seddon"
2,3012,"Brooklyn, Brooklyn, Kingsville, Maidstone, Wes..."
3,3013,"Yarraville, Aintree, Bonnie Brook"
4,3015,"Newport, Spotswood, South Kingsville"
...,...,...
188,3975,"Lynbrook, Lyndhurst"
189,3976,Hampton Park
190,3977,"Botanic Ridge, Cranbourne, Cranbourne East, Cr..."
191,3978,"Clyde, Clyde North"


In [6]:
house_clean_1_df = pd.merge(melb_postcodes_df, house_prices_df, how='inner', on='HOUSE_LOCALITY')
house_clean_1_df = house_clean_1_df.dropna(subset=['HOUSE_LOCALITY'])
columns_to_drop = ["RURAL_TOWNSHIP","ABS_SA2_KEY","HOUSE_LOCALITY","SCHOOL_POST_CODE",
                   "COMMENTS","SUBURB_NAME","SUBURB_POSTCODE_COMMENTS","prelim 2023",
                   "change 2021-22","change 2021-22","Growth PA"]
house_clean_1_df = house_clean_1_df.drop(columns_to_drop, axis=1)
house_clean_1_df

,MUNICIPALITY,CITY_SHIRE,SUBURB_GROUP,POST_CODE,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Inner City municipalities and their suburbs,City of Melbourne,Inner,3053,870000,1002500,1020000,830000,1050000,1470000,1625000,1308000,1452500,1492500,1500000
1,Inner City municipalities and their suburbs,City of Yarra,Inner,3054,940000,890500,1095000,1177500,1360000,1628000,1445000,1555000,1500000,1718500,1645000
2,Inner City municipalities and their suburbs,City of Yarra,Inner,3054,940000,890500,1095000,1177500,1360000,1628000,1445000,1555000,1500000,1718500,1645000
3,Northern municipalities and their suburbs,City of Moonee Valley,Mid,3031,652000,735500,730000,830000,918000,1100000,926500,1000000,1035000,1261500,1103500
4,Northern municipalities and their suburbs,City of Moonee Valley,Mid,3031,650000,690000,737500,823500,908000,1019000,1020000,978000,1106500,1165000,1200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
400,Western municipalities and their suburbs,City of Wyndham,Outer,3029,356000,371000,395000,400000,445000,515000,570000,550000,570000,602500,650000
401,Western municipalities and their suburbs,City of Wyndham,Outer,3030,300000,309000,315000,335000,403000,470000,510000,500000,516000,570000,614000
402,Western municipalities and their suburbs,City of Wyndham,Outer,3030,440000,538000,585000,680000,540000,650000,735000,744500,650000,822500,855000
403,Western municipalities and their suburbs,City of Wyndham,Outer,3024,286500,288000,306000,325000,365000,435500,474000,460500,480000,530000,580000


In [16]:
house_clean_2_df = pd.merge(house_clean_1_df, postcode_suburbs_df, how='inner', on='POST_CODE')
new_column_names = {
    "SUBURB_NAME": "SUBURB_NAMES",
    }
house_clean_2_df = house_clean_2_df.rename(columns=new_column_names)
house_clean_2_df = house_clean_2_df.drop_duplicates()
house_clean_2_df

,MUNICIPALITY,CITY_SHIRE,SUBURB_GROUP,POST_CODE,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,SUBURB_NAMES
0,Inner City municipalities and their suburbs,City of Melbourne,Inner,3053,870000,1002500,1020000,830000,1050000,1470000,1625000,1308000,1452500,1492500,1500000,Carlton
1,Inner City municipalities and their suburbs,City of Yarra,Inner,3054,940000,890500,1095000,1177500,1360000,1628000,1445000,1555000,1500000,1718500,1645000,"Carlton North, Carlton North, Princes Hill"
3,Inner City municipalities and their suburbs,City of Yarra,Inner,3054,928500,1216500,1210000,1280000,1555000,1767500,1650000,2035000,1600000,1992500,1892500,"Carlton North, Carlton North, Princes Hill"
4,Northern municipalities and their suburbs,City of Moonee Valley,Mid,3031,652000,735500,730000,830000,918000,1100000,926500,1000000,1035000,1261500,1103500,"Flemington, Kensington"
5,Northern municipalities and their suburbs,City of Moonee Valley,Mid,3031,650000,690000,737500,823500,908000,1019000,1020000,978000,1106500,1165000,1200000,"Flemington, Kensington"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
400,Western municipalities and their suburbs,City of Wyndham,Outer,3029,312500,321500,340000,365500,415000,516000,552000,526500,550000,585000,620000,"Truganina, Weir Views, Hoppers Crossing, Tarne..."
401,Western municipalities and their suburbs,City of Wyndham,Outer,3029,356000,371000,395000,400000,445000,515000,570000,550000,570000,602500,650000,"Truganina, Weir Views, Hoppers Crossing, Tarne..."
402,Western municipalities and their suburbs,City of Wyndham,Outer,3024,345000,340500,360000,385000,417500,490000,526000,533000,570000,596000,648000,"Manor Lakes, Wyndham Vale, Mambourin"
403,Western municipalities and their suburbs,City of Wyndham,Outer,3024,286500,288000,306000,325000,365000,435500,474000,460500,480000,530000,580000,"Manor Lakes, Wyndham Vale, Mambourin"


In [17]:
unique_count = house_clean_2_df['POST_CODE'].nunique()

# Display the count of unique strings
print(unique_count)

193


In [18]:
# Check for NaN values in a column
has_nan = house_clean_2_df['POST_CODE'].isna().any()

# Display the result - False (no NaN values in column), True (at least one NaN value in column)
print(has_nan)

False


In [19]:
# Print the column names
print(house_clean_2_df.columns)

Index(['MUNICIPALITY',   'CITY_SHIRE', 'SUBURB_GROUP',    'POST_CODE',
                 2012,           2013,           2014,           2015,
                 2016,           2017,           2018,           2019,
                 2020,           2021,           2022, 'SUBURB_NAMES'],
      dtype='object')


In [20]:
# Set the display format for the specified columns
pd.options.display.float_format = "{:,.0f}".format

# Specify the columns to group by
group_cols = ['MUNICIPALITY', 'CITY_SHIRE', 'SUBURB_GROUP', 'POST_CODE']

# Convert columns to numeric types, replacing non-numeric values with NaN
house_clean_2_df[mean_cols] = house_clean_2_df[mean_cols].apply(pd.to_numeric, errors='coerce')

# Calculate the average of each group for the specified columns
house_clean_final_df = house_clean_2_df.groupby(group_cols)[mean_cols].mean().reset_index()

# Remove duplicate post codes
house_clean_final_df = grouped_df.drop_duplicates(subset='POST_CODE')


# Display the grouped DataFrame
house_clean_final_df


,MUNICIPALITY,CITY_SHIRE,SUBURB_GROUP,POST_CODE,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Eastern municipalities and their suburbs,City of Boroondara,Mid,3101,"1,500,000","1,751,000","1,950,000","2,080,000","2,207,500","2,317,500","2,380,000","2,310,000","2,251,500","2,775,000","2,850,000"
1,Eastern municipalities and their suburbs,City of Boroondara,Mid,3102,"1,000,250","1,235,000","1,307,500","1,632,500","1,741,250","1,878,000","1,783,750","1,835,000","1,812,500","2,038,000","2,147,500"
2,Eastern municipalities and their suburbs,City of Boroondara,Mid,3103,"1,585,000","1,755,500","1,997,500","2,480,000","2,390,000","2,601,250","2,556,250","2,617,750","2,705,000","3,000,000","2,925,750"
3,Eastern municipalities and their suburbs,City of Boroondara,Mid,3104,"997,500","1,240,000","1,452,000","1,850,000","1,800,000","1,950,000","1,810,000","1,720,000","1,900,000","2,252,500","2,230,000"
4,Eastern municipalities and their suburbs,City of Boroondara,Mid,3122,"1,405,000","1,575,000","1,606,500","1,916,500","2,245,000","2,650,000","2,506,500","2,200,000","2,512,500","2,915,000","2,475,000"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,Western municipalities and their suburbs,City of Melton,Outer,3338,"385,100","347,100","361,700","363,700","387,600","449,400","511,700","492,000","527,000","577,800","621,000"
189,Western municipalities and their suburbs,City of Melton,Outer,3427,"279,000","280,000","325,000","325,000","360,000","440,500","500,000","515,000","560,000","614,000","650,500"
190,Western municipalities and their suburbs,City of Wyndham,Outer,3024,"315,750","314,250","333,000","355,000","391,250","462,750","500,000","496,750","459,833","574,500","619,833"
191,Western municipalities and their suburbs,City of Wyndham,Outer,3029,"354,250","345,000","378,250","386,375","427,000","497,625","540,500","529,125","548,875","583,375","626,625"


In [21]:
house_clean_final_df = house_clean_final_df.drop_duplicates()
# Reset the index and make POST_CODE the new index
house_clean_final_df = house_clean_final_df.set_index('POST_CODE')

# Export cleaned population dataset to a CSV file
house_clean_final_df.to_csv("Cleaned_Data/house_clean_final.csv", index_label="POST_CODE")
house_clean_final_df


,MUNICIPALITY,CITY_SHIRE,SUBURB_GROUP,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
POST_CODE,,,,,,,,,,,,,,
3101,Eastern municipalities and their suburbs,City of Boroondara,Mid,"1,500,000","1,751,000","1,950,000","2,080,000","2,207,500","2,317,500","2,380,000","2,310,000","2,251,500","2,775,000","2,850,000"
3102,Eastern municipalities and their suburbs,City of Boroondara,Mid,"1,000,250","1,235,000","1,307,500","1,632,500","1,741,250","1,878,000","1,783,750","1,835,000","1,812,500","2,038,000","2,147,500"
3103,Eastern municipalities and their suburbs,City of Boroondara,Mid,"1,585,000","1,755,500","1,997,500","2,480,000","2,390,000","2,601,250","2,556,250","2,617,750","2,705,000","3,000,000","2,925,750"
3104,Eastern municipalities and their suburbs,City of Boroondara,Mid,"997,500","1,240,000","1,452,000","1,850,000","1,800,000","1,950,000","1,810,000","1,720,000","1,900,000","2,252,500","2,230,000"
3122,Eastern municipalities and their suburbs,City of Boroondara,Mid,"1,405,000","1,575,000","1,606,500","1,916,500","2,245,000","2,650,000","2,506,500","2,200,000","2,512,500","2,915,000","2,475,000"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3338,Western municipalities and their suburbs,City of Melton,Outer,"385,100","347,100","361,700","363,700","387,600","449,400","511,700","492,000","527,000","577,800","621,000"
3427,Western municipalities and their suburbs,City of Melton,Outer,"279,000","280,000","325,000","325,000","360,000","440,500","500,000","515,000","560,000","614,000","650,500"
3024,Western municipalities and their suburbs,City of Wyndham,Outer,"315,750","314,250","333,000","355,000","391,250","462,750","500,000","496,750","459,833","574,500","619,833"
